<a href="https://colab.research.google.com/github/nurayyozden/VRP-Problem/blob/main/Plane_VRP_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from datetime import datetime, timedelta
from scipy.optimize import linear_sum_assignment
import math
from scipy.spatial import distance_matrix
import random

In [ ]:
!pip install ortools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# JOBS
jobs = pd.read_csv('aoi_sample_list_194_top_50.csv')

# RESOURCES
planes = pd.read_csv('Plane_Lens_GSD_Capture_Rate_Sample.csv')

In [ ]:
planes.head()

,plane_id,plane_name,tail_num,lens_name,lens_id,gsd,capture_rate,fleet_id,fleet_name
0,34,Pacific Plane 7,N500TD,GE300,1,1.0,6.3,2,Pacific
1,27,Pacific Plane 33,N52EV,GE300,1,1.0,6.3,2,Pacific
2,34,Pacific Plane 7,N500TD,GE300,1,2.0,47.5,2,Pacific
3,27,Pacific Plane 33,N52EV,GE300,1,2.0,47.5,2,Pacific
4,34,Pacific Plane 7,N500TD,GE300,1,3.0,47.5,2,Pacific


In [ ]:
jobs.head()

,aoi_id,opp_number,account_name,city,state,lat,lon,gsd,sq_miles,add_on_qty,...,fleet_id,preferred_lens_id,required_lens_id,revenue,altitude,state_id,aoi_sort,days_in_capture_window,capture_end_date,aoi_list_id
0,273223,C21704329,New York City Department of Information Techno...,New York,NaN,40.713794,-74.011469,2,8946,NaN,...,3,0,0,4249350,1000,4,1,NaN,2022-12-31,194
1,273224,C25571242,"Town of Groveland, NY",Groveland,NaN,42.702415,-77.748764,3,48,NaN,...,2,0,0,19200,1000,4,4,NaN,2022-12-31,194
2,273225,C21704329,New York City Department of Information Techno...,New York,NaN,40.713794,-74.011469,3,994,NaN,...,2,0,0,377720,1000,4,1,NaN,2022-12-31,194
3,273226,C13201173,"Lewis County, NY",Lowville,NaN,43.799200,-75.457700,3,128,NaN,...,2,0,0,46080,1000,4,3,NaN,2022-12-31,194
4,273227,C6866382,"Steuben County, NY",Bath,NaN,42.332939,-77.316605,4,27,NaN,...,3,0,0,9180,1000,4,4,NaN,2022-12-31,194


In [ ]:
# creating distance matrix

# change pracjobs to jobs for all 50 jobs

data = jobs[['lat','lon']].values.tolist()
ctys = jobs['city'].tolist()
coord_df = pd.DataFrame(data, columns=['xcord', 'ycord'], index=ctys)
coord_df.index.name = 'City'

dist_matrix = pd.DataFrame(distance_matrix(coord_df.values, coord_df.values), index=coord_df.index, columns=coord_df.index)

coord_df = coord_df.reset_index()
coord_df = coord_df.reset_index()

In [ ]:
coord_df

,index,City,xcord,ycord
0,0,New York,40.713794,-74.011469
1,1,Groveland,42.702415,-77.748764
2,2,New York,40.713794,-74.011469
3,3,Lowville,43.799200,-75.457700
4,4,Bath,42.332939,-77.316605
5,5,Troy,42.728912,-73.685722
6,6,Bath,42.332939,-77.316605
7,7,Troy,42.728912,-73.685722
8,8,Mahopac,41.377312,-73.723663
9,9,Lowville,43.799200,-75.457700


In [ ]:
dist_matrix_list = dist_matrix.values.tolist()

#1: Minimized Total Distance Across All Planes - standard VRP



In [ ]:
coord_df['path'] = np.nan

In [ ]:

# Getting routes
def get_routes(solution, routing, manager):
  routes = []
  for route_nbr in range(routing.vehicles()):
    index = routing.Start(route_nbr)
    route = [manager.IndexToNode(index)]
    while not routing.IsEnd(index):
      index = solution.Value(routing.NextVar(index))
      route.append(manager.IndexToNode(index))
    routes.append(route)
  return routes

In [ ]:
routes_dict = {}

In [ ]:
"""Simple Vehicles Routing Problem (VRP).

   This is a sample using the routing library python wrapper to solve a VRP
   problem.
   A description of the problem can be found here:
   http://en.wikipedia.org/wiki/Vehicle_routing_problem.

   Distances are in meters.
"""

num_nodes = len(dist_matrix_list) # NUMBER OF JOBS
num_vehicles = 10 # NUMBER OF PLANES



from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = dist_matrix_list
    data['num_vehicles'] = 10
    data['depot'] = 0
    n = random.sample(range(0, 50), 10) # STARTING POINTS
    data['starts'] = n
    data['ends'] = n
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    max_route_distance = 0
    total_route_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        ids = []
        while not routing.IsEnd(index):
            plan_output += ' {} -> '.format(manager.IndexToNode(index))
            ids.append(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        for i in coord_df['index']:
          if i in ids:
            coord_df['path'][i] = vehicle_id
        total_route_distance += route_distance
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
    print('Maximum of the route distances: {}m'.format(max_route_distance))
    print('Total route distance: ' + str(total_route_distance))






def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['starts'],
                                           data['ends'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Distance constraint.
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        3000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)


    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.FromSeconds(5) # Give 5 seconds of search

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)
        routes = get_routes(solution, routing, manager)
        for i, route in enumerate(routes):
          routes_dict["Vehicle " + str(i)] = route
    else:
        print('No solution found !')


if __name__ == '__main__':
    main()

Objective: 625
Route for vehicle 0:
 39 ->  41 ->  40 ->  45 ->  44 -> 39
Distance of the route: 3m

Route for vehicle 1:
 16 ->  25 ->  22 -> 16
Distance of the route: 2m

Route for vehicle 2:
 27 -> 27
Distance of the route: 0m

Route for vehicle 3:
 46 ->  49 ->  48 ->  38 ->  37 ->  31 ->  29 ->  47 ->  32 ->  43 ->  13 -> 46
Distance of the route: 2m

Route for vehicle 4:
 30 -> 30
Distance of the route: 0m

Route for vehicle 5:
 33 ->  4 ->  1 ->  6 ->  36 ->  34 ->  28 ->  12 ->  11 -> 33
Distance of the route: 6m

Route for vehicle 6:
 35 -> 35
Distance of the route: 0m

Route for vehicle 7:
 7 ->  0 ->  2 ->  8 ->  3 ->  9 ->  5 -> 7
Distance of the route: 6m

Route for vehicle 8:
 24 ->  20 ->  19 ->  21 ->  26 ->  18 ->  15 ->  14 ->  23 ->  17 -> 24
Distance of the route: 6m

Route for vehicle 9:
 42 ->  10 -> 42
Distance of the route: 0m

Maximum of the route distances: 6m
Total route distance: 25


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
# PATHS FOR THE PLANES IN A DICTIONARY

print(routes_dict)

{'Vehicle 0': [39, 41, 40, 45, 44, 39], 'Vehicle 1': [16, 25, 22, 16], 'Vehicle 2': [27, 27], 'Vehicle 3': [46, 49, 48, 38, 37, 31, 29, 47, 32, 43, 13, 46], 'Vehicle 4': [30, 30], 'Vehicle 5': [33, 4, 1, 6, 36, 34, 28, 12, 11, 33], 'Vehicle 6': [35, 35], 'Vehicle 7': [7, 0, 2, 8, 3, 9, 5, 7], 'Vehicle 8': [24, 20, 19, 21, 26, 18, 15, 14, 23, 17, 24], 'Vehicle 9': [42, 10, 42]}


In [ ]:
coord_df

,index,City,xcord,ycord,path
0,0,New York,40.713794,-74.011469,7.0
1,1,Groveland,42.702415,-77.748764,5.0
2,2,New York,40.713794,-74.011469,7.0
3,3,Lowville,43.799200,-75.457700,7.0
4,4,Bath,42.332939,-77.316605,5.0
5,5,Troy,42.728912,-73.685722,7.0
6,6,Bath,42.332939,-77.316605,5.0
7,7,Troy,42.728912,-73.685722,7.0
8,8,Mahopac,41.377312,-73.723663,7.0
9,9,Lowville,43.799200,-75.457700,7.0


In [ ]:
pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 4.1 MB/s 
     |████████████████████████████████| 6.3 MB 55.3 MB/s 
     |████████████████████████████████| 16.7 MB 44.8 MB/s 


In [ ]:
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame
import plotly.express as px


#fig = px.scatter_geo(coord_df,lat='xcord',lon='ycord', hover_name='index', scope='usa')
fig = px.line_geo(coord_df, lat='xcord', lon='ycord', scope='usa', color='path')
fig.update_layout(title = 'USA map', title_x=0.5)
fig.show()